In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

In [2]:
seeval = "start_entropy_eval.txt"
setrain = "start_entropy_train.txt"
eeeval = "end_entropy_eval.txt"
eetrain = "end_entropy_train.txt"

nparrsev = np.loadtxt(seeval, delimiter=",")
nparrset = np.loadtxt(setrain, delimiter=",")
nparreev = np.loadtxt(eeeval, delimiter=",")
nparreet = np.loadtxt(eetrain, delimiter=",")

In [3]:
avg = 0 
start_layer = 6

stest = nparrsev[:,:11]
strain = nparrset[:,:11]
etest = nparreev[:,:11]
etrain = nparreet[:,:11]

if avg:
  (x_train_s, y_train_s) = (strain[:,:start_layer], strain[:,start_layer+1:])
  (x_test_s, y_test_s) = (stest[:,:start_layer], stest[:,start_layer+1:])
  (x_train_e, y_train_e) = (etrain[:,:start_layer], etrain[:,start_layer+1:])
  (x_test_e, y_test_e) = (etest[:,:start_layer], etest[:,start_layer+1:])
  x_train_s = np.mean(x_train_s, axis=-1)
  x_test_s = np.mean(x_test_s, axis=-1)
  x_train_e = np.mean(x_train_e, axis=-1)
  x_test_e = np.mean(x_test_e, axis=-1)
else:
  (x_train_s, y_train_s) = (strain[:,start_layer], strain[:,start_layer+1:])
  (x_test_s, y_test_s) = (stest[:,start_layer], stest[:,start_layer+1:])
  (x_train_e, y_train_e) = (etrain[:,start_layer], etrain[:,start_layer+1:])
  (x_test_e, y_test_e) = (etest[:,start_layer], etest[:,start_layer+1:])

In [5]:
smodel = Sequential()
smodel.add(Dense(64, activation='relu', input_dim = 1))
smodel.add(Dense(64, activation='relu'))
smodel.add(Dense(64, activation='relu'))
smodel.add(Dense(64, activation='relu'))
smodel.add(Dense(10-start_layer))

smodel.compile(
    optimizer='adam',
    loss='mean_squared_error' 
)

smodel.fit(x_train_s, y_train_s, epochs=3, verbose=1)

Epoch 1/3
662/662 [==============================] - 2s 2ms/step - loss: 0.0851
Epoch 2/3
662/662 [==============================] - 1s 2ms/step - loss: 0.0706
Epoch 3/3
662/662 [==============================] - 1s 2ms/step - loss: 0.0704


In [6]:
emodel = Sequential()
emodel.add(Dense(64, activation='relu', input_dim = 1))
emodel.add(Dense(64, activation='relu'))
emodel.add(Dense(64, activation='relu'))
emodel.add(Dense(64, activation='relu'))
emodel.add(Dense(10-start_layer))

emodel.compile(
    optimizer='adam',
    loss='mean_squared_error' 
)

emodel.fit(x_train_e, y_train_e, epochs=3, verbose=1)

Epoch 1/3
662/662 [==============================] - 2s 3ms/step - loss: 0.0716
Epoch 2/3
662/662 [==============================] - 1s 2ms/step - loss: 0.0644
Epoch 3/3
662/662 [==============================] - 1s 2ms/step - loss: 0.0638


In [7]:
val_loss_s = smodel.evaluate(x_test_s, y_test_s)
val_loss_e = emodel.evaluate(x_test_e, y_test_e)

9/9 [==============================] - 0s 1ms/step - loss: 0.0597


In [ ]:
table_s = np.zeros((100, 11-start_layer))
table_e = np.zeros((100, 11-start_layer))
for i in range(0,100):
  xval = i*0.01;
  yval_s = smodel.predict(np.array([xval]), batch_size=1)
  yval_e = emodel.predict(np.array([xval]), batch_size=1)
  table_s[i,0] = xval
  table_e[i,0] = xval
  table_s[i,1:] = yval_s[0]
  table_e[i,1:] = yval_e[0]

threshold_indices_s = table_s < 0
threshold_indices_e = table_e < 0
table_s[threshold_indices_s] = 0
table_e[threshold_indices_e] = 0

np.savetxt("squad2_lookup_table_start.csv", table_s, delimiter=",")
np.savetxt("squad2_lookup_table_end.csv", table_e, delimiter=",")